# **Resampling Raster Layers and Preparing Data for Download**
**Last update: 20-Sep-2024**

## Introduction

This notebook helps users resample raster datasets (such as precipitation (PCP) and reference evapotranspiration (RET)) to match the spatial resolution and  coordinate reference system (CRS) of a reference raster (Actual Evapotranspiration and Interception - AETI).

Key steps include:
1. **Importing packages** like `rasterio` and `numpy`.
2. **Uploading data** (raster layers and reference file) from local or Google Drive.
3. **Resampling raster layers** to align with the reference file.
4. **Zipping and downloading** resampled files for further use.

### Data: to be resampled (e.g., PCP, RET) and to be used as a reference (e.g., AETI)

This workflow prepares raster layers for further analysis or integration into models.

## **Step 1 - Import Packages**

In [1]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 22.2 MB/s eta 0:00:00


In [2]:
import os                                        # Module for interacting with the operating system (e.g., file and directory manipulation)
import glob                                      # Used to retrieve files/pathnames matching a specified pattern (e.g., finding all .tif files in a folder)
import rasterio                                  # Library for reading, writing, and analyzing geospatial raster data (e.g., handling .tif files)
from rasterio.warp import reproject, Resampling  # Functions for reprojection and resampling raster data to match spatial properties (e.g., CRS, resolution)
import numpy as np                               # Library for numerical computations, used here for handling array data (e.g., raster data as arrays)

## **Step 2 - Import the files**

In [3]:
# Upload the layers to be resampled
from google.colab import files
uploaded = files.upload()

Saving PCPRETdata.zip to PCPRETdata.zip


In [4]:
#Unzip
!unzip '/content/PCPRETdata.zip' -d '/content'

Archive:  /content/PCPRETdata.zip
   creating: /content/L1-PCP-D/
   creating: /content/L1-RET-D/
  inflating: /content/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-04-01.tif  
  inflating: /content/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-01-11.tif  
  inflating: /content/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-04-21.tif  
  inflating: /content/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-02-11.tif  
  inflating: /content/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-04-11.tif  
  inflating: /content/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-01-01.tif  
  inflating: /content/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2022-12-01.tif  
  inflating: /content/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2022-11-21.tif  
  inflating: /content/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-02-01.tif  
  inflating: /content/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2022-11-11.tif  
  inflating: /content/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2022-12-11.tif  
  inflating

In [5]:
# Upload the the reference layer for resampling, seasonal AETI, which is output of Module_1_Seasonal_AETI...
from google.colab import files
uploaded = files.upload()

Saving WaPORv3_M1_Seasonal.zip to WaPORv3_M1_Seasonal.zip


In [6]:
#Unzip
!unzip '/content/WaPORv3_M1_Seasonal.zip' -d '/content'

Archive:  /content/WaPORv3_M1_Seasonal.zip
   creating: /content/content/output_M1a/AETI_season/
  inflating: /content/content/output_M1a/AETI_season/AETI_season1_2022-10-01_to_2023-04-30.tif  
   creating: /content/content/output_M1a/NPP_season/
  inflating: /content/content/output_M1a/NPP_season/NPP_season1_2022-10-01_to_2023-04-30.tif  
   creating: /content/content/output_M1a/T_season/
  inflating: /content/content/output_M1a/T_season/T_season1_2022-10-01_to_2023-04-30.tif  


##  Define project directory and the location of WaPOR data

In [7]:
# Paths to the files
dir_proj     = os.path.split(os.getcwd())[0]
dir_data     = "/content/"

input_folderPCP = os.path.join(dir_proj, dir_data, "L1-PCP-D")
input_folderRET = os.path.join(dir_proj, dir_data, "L1-RET-D")

et_files        = os.path.join(dir_proj, dir_data, "content/output_M1a/AETI_season")
et_file         = sorted(glob.glob(et_files  + '/*.tif'))[0]

input_folderPCP, input_folderRET, et_file

('/content/L1-PCP-D',
 '/content/L1-RET-D',
 '/content/content/output_M1a/AETI_season/AETI_season1_2022-10-01_to_2023-04-30.tif')

## **Step 3 - Resample the layers**

In [8]:
# Function for resampling

def resample_raster_files(input_folder, reference_file, output_folder):
    """
    Resamples all raster files in the input folder to match the spatial resolution and CRS
    of the reference raster file and saves the resampled files in the output folder.

    Parameters:
    input_folder (str): Path to the folder containing the raster files to be resampled.
    reference_file (str): Path to the reference raster file used for resampling.
    output_folder (str): Path to the folder where the resampled files will be saved.
    """

    # Get all raster files to be resampled
    input_files = glob.glob(os.path.join(input_folder, "*.tif"))

    # Open the reference file to get its profile
    with rasterio.open(reference_file) as ref_src:
        ref_profile = ref_src.profile

        # Loop through each raster file to be resampled
        for input_file in input_files:
            with rasterio.open(input_file) as src:
                src_data = src.read(1)

                # Create an empty array to store the resampled data
                resampled_data = np.empty((ref_profile['height'], ref_profile['width']), dtype=src_data.dtype)

                # Resample the data
                reproject(
                    source=src_data,
                    destination=resampled_data,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=ref_profile['transform'],
                    dst_crs=ref_profile['crs'],
                    resampling=Resampling.nearest
                )

                # Update profile metadata using the reference file's profile
                output_profile = src.profile.copy()
                output_profile.update(
                    transform=ref_profile['transform'],
                    height=ref_profile['height'],
                    width=ref_profile['width'],
                    crs=ref_profile['crs']
                )

                # Preserve band-specific metadata
                band1_meta = src.tags(1)

                # Save resampled file with the updated profile and original metadata
                output_file = os.path.join(output_folder, os.path.basename(input_file))
                with rasterio.open(output_file, 'w', **output_profile) as dst:
                    dst.write(resampled_data, 1)
                    dst.update_tags(1, **band1_meta)  # Preserve band metadata

    print("Resampling completed.")

# Example usage:
# resample_raster_files(input_folder="path/to/input/folder", reference_file="path/to/reference/file.tif", output_folder="path/to/output/folder")


In [10]:
# Resample pricipitation
input_folder   = input_folderPCP
reference_file = et_file
output_folder  = os.path.join(dir_proj, dir_data, "PCP_resampled")  #  the directory of the output folder

# Make one if the folder does not exit
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

resample_raster_files (input_folder, reference_file, output_folder)

Resampling completed.


In [11]:
# Resample reference evapotranspiration
input_folder   = input_folderRET
reference_file = et_file
output_folder  = os.path.join(dir_proj, dir_data, "RET_resampled")  #  the directory of the output folder

# Make one if the folder does not exit
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

resample_raster_files (input_folder, reference_file, output_folder)

Resampling completed.


## **Step 4 - Zip the resampled raster files and download them to the local folder**

In [12]:
import shutil
from google.colab import files
import os

# 1) Specify the directories for PCP and RET resampled data
pcp_resampled_directory = r'/content/PCP_resampled'
ret_resampled_directory = r'/content/RET_resampled'

# 2) Create a new folder to temporarily store both PCP and RET resampled data in separate subfolders
combined_directory = r'/content/PCPRET_combined'
os.makedirs(combined_directory, exist_ok=True)

# Create separate subfolders for PCP and RET within the combined directory
pcp_output_folder = os.path.join(combined_directory, 'PCP_resampled')
ret_output_folder = os.path.join(combined_directory, 'RET_resampled')

os.makedirs(pcp_output_folder, exist_ok=True)
os.makedirs(ret_output_folder, exist_ok=True)

# Copy PCP resampled files into the PCP folder
for file_name in os.listdir(pcp_resampled_directory):
    full_file_name = os.path.join(pcp_resampled_directory, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, pcp_output_folder)

# Copy RET resampled files into the RET folder
for file_name in os.listdir(ret_resampled_directory):
    full_file_name = os.path.join(ret_resampled_directory, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, ret_output_folder)

# 3) Zip the combined directory into "PCPRETdata_resampled.zip"
zip_filename = r'/content/PCPRETdata_resampled.zip'
shutil.make_archive('/content/PCPRETdata_resampled', 'zip', combined_directory)

# 4) Download the zipped folder
files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>